In [85]:
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinless_fermion_basis_1d,boson_basis_1d, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [99]:
L = 3
M = 0.1
l=1/2
m_epsilon = 1/2
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [100]:
basis_spin =spin_basis_1d(L=L,S = "1/2",pauli = 0)
basis_fermions = spinless_fermion_basis_1d(L=L+1,Nf = [0,1,2,3,4])
basis=tensor_basis(basis_spin,basis_fermions)
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
	  0.         |1 1 1>           7         |1 1 1 1>           15  
	  1.         |1 1 1>           7         |1 1 1 0>           14  
	  2.         |1 1 1>           7         |1 1 0 1>           13  
	  3.         |1 1 1>           7         |1 1 0 0>           12  
	  4.         |1 1 1>           7         |1 0 1 1>           11  
	  5.         |1 1 1>           7         |1 0 1 0>           10  
	  6.         |1 1 1>           7         |1 0 0 1>            9  
	  7.         |1 1 1>           7         |1 0 0 0>            8  
	  8.         |1 1 1>           7         |0 1 1 1>            7  
	  9.         |1 1 1>           7         |0 1 1 0>            6  
	 10.         |1 1 1>           7         |0 1 0 1>            5  
	 11.         |1 1 1>           7         |0 1 0 0>            4  
	 12.         |1 1 1>           7         |0 0 1 1>            3  
	 13.         |1 1 1>           7         |0 0 1 0>         

In [101]:
#initialisation of the Gauss laws

liste = []
liste_epsilon = [[m_epsilon,0]]
liste_epsilon_m = [[-m_epsilon,L-1]]

liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z|",liste[0]],
            ["|n",liste_m[0]],
            ["I|",liste_epsilon_m],
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]],
            ["I|",liste_epsilon],

        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["I|",liste[i-1]],
            ["|n",liste_m[i]],
            ["I|",liste_epsilon],

        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],  
            ["I|",liste[i]],
            ["|n",liste_m[i]],
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
G = 0
for i in range(L+1):
    G += Gauss_law[i]**2

In [102]:
kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[M*(-1)**i,i] for i in range(L+1)]
interaction_term = [[-1j/(2*np.sqrt(l*(l+1))),i,i,i+1] for i in range(L)]

hamiltonian_map = [
    
    ["zz|",kin_energy],
    ["|n",mass_term],
    ["|I",mass_term],

    ["+|+-",interaction_term],
    ["-|-+",interaction_term]
]



Ham_LGT = hamiltonian(hamiltonian_map,dynamic_list = [],basis=basis,**no_checks)
Hamiltonian_LGT = Ham_LGT + 1e2*G
ham_matrix = Hamiltonian_LGT.toarray()
ham_LGT_matrix = Ham_LGT.toarray()

In [103]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [104]:
print(check_hermitian(ham_matrix))

True


In [105]:
print(check_hermitian(ham_LGT_matrix))

True


In [106]:
eigenval,eigenvec = np.linalg.eigh(ham_LGT_matrix)

In [107]:
print(eigenval)

[-0.89858083 -0.85163135 -0.56450944 -0.56450944 -0.54759751 -0.54759751
 -0.44759751 -0.44759751 -0.44759751 -0.44759751 -0.34759751 -0.34759751
 -0.31094653 -0.31094653 -0.31094653 -0.21094653 -0.21094653 -0.21094653
 -0.21094653 -0.21094653 -0.21094653 -0.21094653 -0.21094653 -0.21094653
 -0.21094653 -0.11094653 -0.11094653 -0.11094653 -0.0263627   0.00443016
  0.00443016  0.02058679  0.175       0.275       0.275       0.275
  0.275       0.275       0.275       0.275       0.275       0.275
  0.275       0.275       0.275       0.275       0.275       0.375
  0.375       0.375       0.375       0.375       0.375       0.375
  0.375       0.375       0.375       0.375       0.375       0.375
  0.375       0.375       0.375       0.375       0.375       0.375
  0.375       0.375       0.375       0.375       0.375       0.375
  0.375       0.375       0.375       0.375       0.375       0.375
  0.375       0.375       0.375       0.475       0.475       0.475
  0.475       0.475    

In [108]:
eigenvalues,eigenvectors = np.linalg.eigh(ham_matrix)

In [109]:
print(eigenvalues[eigenvalues<=1e1])

[-0.31094653  0.86094653]


In [97]:
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,3][i])>=1e-3:
        print(i)
        print(np.abs(eigenvectors[:,3][i]))

35
0.5522908581132391
151
0.26517468367576486
499
0.2651746836757699
615
0.3895102774670618
847
0.634524488153915


In [98]:
print(eigenvectors[:,0])

[-3.01390082e-19+0.00000000e+00j -3.48782704e-21-3.06078181e-20j
  8.29220115e-17-7.14120283e-18j ...  2.79683078e-36-9.76434143e-39j
 -5.95747756e-27+2.25512699e-21j  0.00000000e+00+0.00000000e+00j]
